# 📚 Library Management System
A professional system for managing library operations including book records, member transactions, and inventory analytics using SQLite and Python.

### 🗝️ Core Features:
- **Database Management**: Robust SQLite integration for persistent storage.
- **Transaction Tracking**: Recording issue and return operations.
- **Inventory Analytics**: visualizing book distribution and popular genres.
- **Interactive Interface**: Using `ipywidgets` for on-the-fly management.

In [ ]:
import sqlite3
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output

# Database Initialization
def init_db():
    conn = sqlite3.connect('library.db')
    cursor = conn.cursor()
    
    # Create Books Table
    cursor.execute('''CREATE TABLE IF NOT EXISTS Books (
                        BookID INTEGER PRIMARY KEY AUTOINCREMENT,
                        Title TEXT, Author TEXT, Genre TEXT, Status TEXT)''')
    
    # Create Members Table
    cursor.execute('''CREATE TABLE IF NOT EXISTS Members (
                        MemberID INTEGER PRIMARY KEY AUTOINCREMENT,
                        Name TEXT, Email TEXT)''')
    
    # Insert Sample Data if empty
    cursor.execute("SELECT count(*) FROM Books")
    if cursor.fetchone()[0] == 0:
        books = [
            ('The Great Gatsby', 'F. Scott Fitzgerald', 'Classic', 'Available'),
            ('1984', 'George Orwell', 'Dystopian', 'Issued'),
            ('Clean Code', 'Robert C. Martin', 'Technical', 'Available'),
            ('Thinking Fast and Slow', 'Daniel Kahneman', 'Psychology', 'Available')
        ]
        cursor.executemany("INSERT INTO Books (Title, Author, Genre, Status) VALUES (?,?,?,?)", books)
    
    conn.commit()
    conn.close()

init_db()
print("Database Initialized Successfully!")

### 📊 1. Inventory Insights

In [ ]:
def show_inventory_chart():
    conn = sqlite3.connect('library.db')
    df = pd.read_sql_query("SELECT Genre, count(*) as Count FROM Books GROUP BY Genre", conn)
    conn.close()
    
    fig = px.pie(df, values='Count', names='Genre', title='Book Distribution by Genre', 
                 hole=0.3, color_discrete_sequence=px.colors.sequential.RdBu)
    fig.update_layout(template='plotly_dark')
    fig.show()

show_inventory_chart()

### 🛠️ 2. Management Dashboard

In [ ]:
def get_all_books():
    conn = sqlite3.connect('library.db')
    df = pd.read_sql_query("SELECT * FROM Books", conn)
    conn.close()
    return df

output = widgets.Output()

def refresh_view(b=None):
    with output:
        clear_output()
        display(get_all_books())

refresh_btn = widgets.Button(description="Refresh Inventory", button_style='info')
refresh_btn.on_click(refresh_view)

display(refresh_btn, output)
refresh_view()